# Step 1: Downloading ArcticDEM Strip Files from PGC

This script automates the process of downloading **ArcticDEM Strip DEM files** using metadata provided in a CSV file. 


### Loads a CSV File
- Reads `csv/ArcticDEM_Stripfiles_Basin.csv`, which contains DEM metadata and file paths

### Constructs Download URLs
- Base URL: `https://pgc-opendata-dems.s3.us-west-2.amazonaws.com/arcticdem/`
- Builds full URLs for each `_dem.tif` file by manipulating the `fileurl` column

### Sets Up Local Download Directory
- Saves downloaded files in the `download_ArcticDEM_Stripfiles` folder

### Downloads Files with Checks
- Skips download if the file already exists and is larger than 10 MB
- Uses `requests.get()` to stream and save each file
- Handles and logs errors

---

In [2]:
import os
import requests
import pandas as pd

In [ ]:
# Load the CSV file
file_path = 'csv/ArcticDEM_Stripfiles_Basin.csv'
arcticdem_csv = pd.read_csv(file_path)

# Base URL
pgc_url_base = 'https://pgc-opendata-dems.s3.us-west-2.amazonaws.com/arcticdem/'

# Extract file paths and form new URLs
arcticdem_csv['pgc_url'] = pgc_url_base + arcticdem_csv['fileurl'].apply(lambda x: '/'.join(x.split('/')[7:])).str[:-7] + '_dem.tif'

# Create a folder to store the downloaded files
output_folder = '1_download'

# Download each file
for i, row in arcticdem_csv.iterrows():
    file_url = row['pgc_url']
    file_name = os.path.join(output_folder, os.path.basename(file_url))
    
    # Check if the file exists and is larger than 10MB
    if os.path.exists(file_name) and os.path.getsize(file_name) > 10 * 1024 * 1024:
        print(f"Skipping (already downloaded and >10MB): {file_name}")
        continue  # Skip this file
    
    try:
        print(f"Downloading: {file_url}")
        response = requests.get(file_url, stream=True)
        response.raise_for_status()  # Check if the request was successful
        
        # Save the file locally
        with open(file_name, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Saved: {file_name}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {file_url}: {e}")